# Name: Falak Jain

# Problem Set 8

### Learning Objective:

- Create Python code to automate a given task.

### Overview:

This problem set assesses your ability to solve linear optimization models using Gurobi and Python, as discussed in the lectures for Week 10.

### Grading

There are three possible scores you can get from submitting this assignment on time (submitting a blank file or one without any apparent effort does not count). Note that the rubric is designed to incentivize you to go for 100% mastery of the material, as the little details matter a lot in business analytics. 

| Grade | Description |
|--|--|
| 5 out of 5 | Perfect submission with no significant errors. | 
| 4 out of 5 | Near perfect submission with one or more significant errors. |
| 2 out of 5 | Apparent effort but far from perfect. |

## Q1. Numerical Solution for Sample Problem 9.3

The concrete formulation of Sample Problem 9.3 is reproduced below:

**Decision Variables:** Let $X_1, \cdots, X_7$ denote whether to use each FC. (Binary)

**Objective and constraints:**

$$\begin{aligned}
\text{Minimize} && X_1+X_2+\cdots+X_7 \\
\text{s.t.} && X_2+X_5+X_6+X_7 & \ge 1\\
&& X_3+X_4 & \ge 1\\
&& X_3 & \ge 1 \\
&& X_1+X_2+X_4+X_6 & \ge 1 \\
&& X_5 + X_7 & \ge 1\\
&& X_4 &\le X_1 \\
&& X_2+X_3 & \le 1
\end{aligned}$$

**a)** Implement the above using Gurobi, while using for loops and list comprehensions as much as possible to automate recurring patterns.

After you are done, use `mod.write`, and `%cat` in Mac or `!type` in Windows to output what the linear optimization formulation looks like according to Gurobi, following Section 10.4. You can use this to verify that you have indeed implemented the above.

In [5]:
# Write your code here
from gurobipy import Model, GRB
mod = Model()
FCs = range(1,8)
x = mod.addVars(FCs,name='x',vtype=GRB.BINARY)
mod.setObjective(sum(x[f] for f in FCs))
mod.addConstr(x[2]+x[5]+x[6]+x[7]>=1)
mod.addConstr(x[3]+x[4]>=1)
mod.addConstr(x[3]>=1)
mod.addConstr(x[1]+x[2]+x[4]+x[6]>=1)
mod.addConstr(x[5]+x[7]>=1)
mod.addConstr(x[4]<=x[1])
mod.addConstr(x[2]+x[3]<=1)
mod.write('10-books.lp')
!type 10-books.lp


\ LP format - for model browsing. Use MPS format to capture full model detail.
Minimize
  x[1] + x[2] + x[3] + x[4] + x[5] + x[6] + x[7]
Subject To
 R0: x[2] + x[5] + x[6] + x[7] >= 1
 R1: x[3] + x[4] >= 1
 R2: x[3] >= 1
 R3: x[1] + x[2] + x[4] + x[6] >= 1
 R4: x[5] + x[7] >= 1
 R5: - x[1] + x[4] <= 0
 R6: x[2] + x[3] <= 1
Bounds
Binaries
 x[1] x[2] x[3] x[4] x[5] x[6] x[7]
End


In [1]:
# Sample Output from %cat or !type

Using license file /home/pengshi/gurobi.lic
Academic license - for non-commercial use only
\ LP format - for model browsing. Use MPS format to capture full model detail.
Minimize
  X[1] + X[2] + X[3] + X[4] + X[5] + X[6] + X[7]
Subject To
 R0: X[2] + X[5] + X[6] + X[7] >= 1
 R1: X[3] + X[4] >= 1
 R2: X[3] >= 1
 R3: X[1] + X[2] + X[4] + X[6] >= 1
 R4: X[5] + X[7] >= 1
 R5: - X[1] + X[4] <= 0
 R6: X[2] + X[3] <= 1
Bounds
Binaries
 X[1] X[2] X[3] X[4] X[5] X[6] X[7]
End


**b)** Solve the MIP and print the minimum number of FCs needed, as well as where to stock the items. The output format must match the sample output below.

In [9]:
# Write your code here
from gurobipy import Model, GRB
mod = Model()
FCs = range(1,8)
x = mod.addVars(FCs,name='x',vtype=GRB.BINARY)
mod.setObjective(sum(x[f] for f in FCs))
mod.addConstr(x[2]+x[5]+x[6]+x[7]>=1)
mod.addConstr(x[3]+x[4]>=1)
mod.addConstr(x[3]>=1)
mod.addConstr(x[1]+x[2]+x[4]+x[6]>=1)
mod.addConstr(x[5]+x[7]>=1)
mod.addConstr(x[4]<=x[1])
mod.addConstr(x[2]+x[3]<=1)
mod.setParam('OutputFlag',False)
mod.optimize()
print('Minimum # of FCs needed:',mod.objVal)
print('Stock item in the following:')
for f in FCs:
    if x[f].x == 1:
        print(f'\tFC{f}')

Minimum # of FCs needed: 3.0
Stock item in the following:
	FC1
	FC3
	FC7


In [2]:
# Sample output

Minimum # of FCs needed: 3
Stock item in the following:
	FC1
	FC3
	FC7


## Q2.Optimizing Production Quantities

**(Adapted from DMD Exercise 7.8)** Nature's Best Frozen Foods company produces four different mixes of frozen ready-to-eat vegetables. The mixes consist of five different vegetables: carrots, mushrooms, green peppers, broccoli, and corn. The company manufacturers four different mixes, each sold in 10 oz. bags. The mixes are "Stir Fry", "Barbecue", "Hearty Mushrooms" and "Veggie Crunch," and their contributions to earnings (per bag) are 0.22, 0.20, 0.18 and 0.18 respectively. The monthly supplies of carrots, mushrooms, green peppers, broccoli and corn are 150,000 oz., 80,000 oz., 135,000 oz., 140,000 oz., and 150,000 oz. per month, respectively. The compositions of the mixes are shown in the table below. For example, one bag of "Stir Fry" mix contains 2.5 oz. of carrots, 3.0 oz. of mushrooms, 2.5 oz. of green peppers, 2.0 oz. of broccoli, and no corn. 

|` `| Stir Fry | Barbecue | Hearty Mushrooms | Veggie Crunch |
|--|--|--|--|--|
|Carrots | 2.5 | 2.0 | - | 2.5 |
|Mushrooms | 3.0 | - | 4.0 | - |
|Green Peppers | 2.5 | 2.0 | 3 | 2.5 |
|Broccoli | 2.0 | 3.0 | 3.0 | 2.5 |
|Corn | - | 3.0 | - | 2.5 |

**a)** Formulate a LP to determine the optimal monthly production of each mix to maximize total earnings. (It is okay to have factional monthly productions.)

### Step 1. English description

**Decision:** 
Determine the optimum monthly production of each mix

**Objective:** 
Maximize total Earnings

**Constraints:**
- The monthly supply of vegetables cannot be exhausted
- Each bag of a mix should consist of the specified quantity of each vegetable and should weigh 10oz


### Step 2. Concrete formulation

**Decision variables:** 
Let $X_{Stir Fry} + \cdots + X_{Veggie Crunch}$ denote the number of bags produced for each mix . (Continuous)

**Objective and constraints:**

$$
\text{Maximize: } \qquad .22X_{Stir Fry}+ .2X_{Barbeque}+ .18X_{Hearty Mushrooms}+ .18X_{Veggie Crunch}
$$

$$
\begin{aligned}
\text{s.t.} \\
&& 2.5X_{Stir Fry}+2X_{Barbeque}+2.5X_{Veggie Crunch} & \le 150000\\
&& 3X_{Stir Fry}+4X_{Hearty Mushrooms} & \le 80000\\
&& 2.5X_{Stir Fry}+2X_{Barbeque}+3X_{Hearty Mushrooms}+2.5X_{Veggie Crunch} & \le 135000 \\
&& 2X_{Stir Fry}+3X_{Barbeque}+3X_{Hearty Mushrooms}+2.5X_{Veggie Crunch} & \le 140000 \\
&& 3X_{Barbeque}+2.5X_{Veggie Crunch} & \le 150000 \\
\text{(Non-negativity)} && X_{Stir Fry},X_{Barbeque},X_{Hearty Mushrooms},X_{Veggie Crunch} & \ge 0 \\
\end{aligned}
$$


**b)** Write Gurobi code to implement the above LP. Your code should read in the data from the following data structures rather than hard code in the numbers. The outputs should all be rounded to two decimal places, as in the sample outputs at the end.

In [44]:
products=['Stir Fry','Barbeque','Hearty Mushrooms','Veggie Crunch']
import pandas as pd
earnings=pd.Series([.22,.2,.18,.18],index=products)
earnings

Stir Fry            0.22
Barbeque            0.20
Hearty Mushrooms    0.18
Veggie Crunch       0.18
dtype: float64

In [45]:
ingredients=['Carrots','Mushrooms','Green peppers','Broccoli','Corn']
supply=pd.Series([150000,80000,135000,140000,150000],index=ingredients)
supply

Carrots          150000
Mushrooms         80000
Green peppers    135000
Broccoli         140000
Corn             150000
dtype: int64

In [46]:
usage=pd.DataFrame([[2.5,2,0,2.5],\
                    [3,0,4,0],\
                    [2.5,2,3,2.5],\
                    [2,3,3,2.5],\
                    [0,3,0,2.5]],\
                  index=ingredients, columns=products)
usage

,Stir Fry,Barbeque,Hearty Mushrooms,Veggie Crunch
Carrots,2.5,2,0,2.5
Mushrooms,3.0,0,4,0.0
Green peppers,2.5,2,3,2.5
Broccoli,2.0,3,3,2.5
Corn,0.0,3,0,2.5


In [49]:
# Write your code here
from gurobipy import Model, GRB

vegetables = usage.index
mod = Model()
x = mod.addVars(products,name = 'x')
mod.setObjective(sum(earnings.loc[i]*x[i] for i in products),sense = GRB.MAXIMIZE)
for i in vegetables:
    mod.addConstr(sum(usage.loc[i,a]*x[a] for a in products)<=supply.loc[i])
mod.setParam('OutputFlag',False)
mod.optimize()
print('Maximum earning:',round(mod.objVal,2))
print('Production Plan')
for i in products:
    print(f'\t{i}: {round(x[i].x,2)}')

Maximum earning: 11813.33
Production Plan
	Stir Fry: 26666.67
	Barbeque: 18333.33
	Hearty Mushrooms: 0.0
	Veggie Crunch: 12666.67


In [6]:
# Sample Output

Maximum earning: 11813.33
Production plan:
	Stir Fry: 26666.67
	Barbeque: 18333.33
	Hearty Mushrooms: 0.0
	Veggie Crunch: 12666.67


## Q3. Optimal Advertising Plan

SALS Marketing Inc. is developing an advertising campaign for a large consumer goods corporation. An advertising plan specifies how many units of each kind of advertisement to purchase. SALS has promised a plan that will yield the highest possible “exposure rating,” which is a measure of the ability to reach the appropriate demographic group and generate demand. The options for advertisements with their respective costs (per unit of advertising) and per-unit exposure ratings are given in the table below (K stands for thousands).

| Category | Subcategory | Cost/Unit | Exposure/Unit |
|--|--|--|--|
| Magazines | Literary | \$7.5 K | 15 K |
| ` ` | News | \$10 K | 22.5 K |
| ` ` | Topical | \$15 K | 24 K |
| Newspapers |  Morning | \$2 K | 37.5 K |
|` `  | Evening | \$3 K | 75 K |
| Television | Morning | \$20 K | 275 K |
| ` ` | Midday | \$10 K | 180 K |
| ` `  | Evening | \$60 K | 810 K |
| Radio | Morning | \$15 K | 180 K |
| ` ` | Midday | \$15 K | 17 K |
| ` ` | Evening | \$10 K | 16 K |

Of course, certain restrictions exist for the advertising campaign. The client corporation has budgeted 800,000 dollars for the campaign, but to restrict overexposure to any particular audience it wants no more than 300,000 dollars put into any one category (Magazine, Newspaper, etc.). Also, to ensure a broad range of exposure, at least 100,000 dollars must be spent in each category. Finally, one has to purchase an integer number of units of each kind of advertisement, as no fractional units are allowed.

**a)** Formulate a linear optimization problem to determine the optimal advertising plan.

### Step 1. English Description

**Decision:** Determine the number of units for each subcategory to be purchased

**Objective:** Maximize the exposure rating of the advertisement campaign

**Constraints:**
- The expenditure in a certain category should not be lower than 100,000 or greater than 300,000
- The expenditure on the advertisement campaign must not exceed the budgeted amount

### Step 2. Concrete Formulation

**Decision variables:** 
Let $X_{Literary Mag.} + \cdots + X_{Evening Radio}$ denote the number of bags produced for each mix . (Integer)

**Objective and constraints:**

$$
\text{Maximize: } \qquad 15X_{Literary Mag.}+22.5X_{News Mag.}+24X_{Topical Mag.}+37.5X_{Morning News}+75X_{Evening News}+275X_{Morning TV}+180X_{Midday TV}+810X_{Evening TV}+180X_{Morning Radio}+17X_{Midday Radio}+16X_{Evening Radio}
$$

$$
\begin{aligned}
\text{s.t.} \\
&& 7.5X_{Literary Mag.} + \cdots + 10X_{Evening Radio} & \le 800\\
&& 7.5X_{Literary Mag.}+10X_{News Mag.}+15X_{Topical Mag.} & \ge 100\\
&& 7.5X_{Literary Mag.}+10X_{News Mag.}+15X_{Topical Mag.} & \le 300\\
&& 2X_{Morning News}+3X_{Evening News} & \ge 100\\
&& 2X_{Morning News}+3X_{Evening News} & \le 300 \\
&& 20X_{Morning TV}+10X_{Midday TV}+60X_{Evening TV} & \ge 100 \\
&& 20X_{Morning TV}+10X_{Midday TV}+60X_{Evening TV} & \le 300 \\
&& 15X_{Morning Radio}+15X_{Midday Radio}+10X_{Evening Radio} & \ge 100 \\
&& 15X_{Morning Radio}+15X_{Midday Radio}+10X_{Evening Radio} & \le 300 \\
\text{(Non-negativity)} && X_{Literary Mag.},X_{News Mag.},X_{Topical Mag.} & \ge 0\\
\text{(Non-negativity)} && X_{Morning News},X_{Evening News} & \ge 0\\
\text{(Non-negativity)} && X_{Morning TV},X_{Midday TV},X_{Evening TV} & \ge 0\\
\text{(Non-negativity)} && X_{Morning Radio},X_{Midday Radio},X_{Evening Radio} & \ge 0 \\
\end{aligned}
$$

**b)** Write Gurobi code to implement the above formulation. Your code should read in the data from the following data structures rather than hard code in the numbers. For convenience, all numerical values are in the units of K (thousands). 

The outputs should be in the same format as the sample outputs below. Note: Gurobi might output strangely formatted numbers like `-0`, and you can make it `0` by converting it to `int`.

In [1]:
# Constructing the subcategories
subcat={}
subcat['Magazines']=['Literary Mag.','News Mag.','Topical Mag.']
subcat['Newspapers']=['Morning News','Evening News']
subcat['Television']=['Morning TV','Midday TV','Evening TV']
subcat['Radio']=['Morning Radio','Midday Radio','Evening Radio']
subcat

{'Magazines': ['Literary Mag.', 'News Mag.', 'Topical Mag.'],
 'Newspapers': ['Morning News', 'Evening News'],
 'Television': ['Morning TV', 'Midday TV', 'Evening TV'],
 'Radio': ['Morning Radio', 'Midday Radio', 'Evening Radio']}

In [8]:
import pandas as pd
allSubCat=subcat['Magazines']+subcat['Newspapers']+subcat['Television']+subcat['Radio']
data=pd.DataFrame([[7.5,15],[10,22.5],[15,24],\
                 [2,37.5],[3,75],
                 [20,275],[10,180],[60,810],\
                 [15,180],[15,17],[10,16]],\
                 index=allSubCat,columns=['Cost','Exposure'])
data

,Cost,Exposure
Literary Mag.,7.5,15.0
News Mag.,10.0,22.5
Topical Mag.,15.0,24.0
Morning News,2.0,37.5
Evening News,3.0,75.0
Morning TV,20.0,275.0
Midday TV,10.0,180.0
Evening TV,60.0,810.0
Morning Radio,15.0,180.0
Midday Radio,15.0,17.0


In [42]:
# Write your code here
from gurobipy import Model, GRB
mod = Model()
x = mod.addVars(data.index,name='x',vtype=GRB.INTEGER)
mod.setObjective(sum(data.loc[a,'Exposure']*x[a] for a in allSubCat),sense = GRB.MAXIMIZE)
mod.addConstr(sum(data.loc[a,'Cost']*x[a] for a in allSubCat)<=800)
for key in subcat.keys():
    mod.addConstr(sum(data.loc[i,'Cost']*x[i] for i in subcat[key])<=300)
    mod.addConstr(sum(data.loc[i,'Cost']*x[i] for i in subcat[key])>=100)
mod.setParam('OutputFlag',False)
mod.optimize()
print('Maximum total exposure (in thousands):',mod.objVal)
print('# of units to purchase:')
for a in allSubCat:
    print(f'\t{a}: {int(x[a].x)}')

Maximum total exposure (in thousands): 14235.0
# of units to purchase:
	Literary Mag.: 0
	News Mag.: 10
	Topical Mag.: 0
	Morning News: 0
	Evening News: 98
	Morning TV: 0
	Midday TV: 30
	Evening TV: 0
	Morning Radio: 7
	Midday Radio: 0
	Evening Radio: 0


In [9]:
# Sample Output

Maximum total exposure (in thousands): 14235.0
# of units to purchase:
	Literary Mag.: 0
	News Mag.: 10
	Topical Mag.: 0
	Morning News: 0
	Evening News: 98
	Morning TV: 0
	Midday TV: 30
	Evening TV: 0
	Morning Radio: 7
	Midday Radio: 0
	Evening Radio: 0
